In [ ]:
# Imports
import numpy as np
import pickle
import scipy.stats as ss
import gurobipy as gp
from gurobipy import GRB

### Bin Packing

In [ ]:
def DiscreteRand(mean, std, lb, ub):
    '''Sample from a discretized gaussian distribution truncated to [lb,ub].'''
    x = np.arange(lb, ub+1)
    xU, xL = x + 0.5, x - 0.5 
    prob = ss.norm.cdf(xU, loc=mean ,scale = std) - ss.norm.cdf(xL, loc=mean, scale = std)
    prob = prob / prob.sum() #normalize the probabilities so their sum is 1
    return np.random.choice(x, p = prob)

In [ ]:
def PerfectBinPacking(mean, std, n, v):
    '''Create a perfect bin packing problem with piece size ~N(mean,std) with n bins of volume v'''
    items = {}
    for bins in range(n):
        b = v
        while b > 0:
            if b >= mean:
                item = DiscreteRand(mean,std,0,b)
            else:
                item = b
            b -= item
            if item in items.keys():
                items[item] += 1
            else:
                items.update({item : 1})    
    return items

In [ ]:
def PackBins(items, volume, integer = False):
    '''A model for a bin packing problem.
    
    Args:
        items (Dict[int,int]): Dictionary from item size to number of those items
        volume (int): Volume of bins.
        integer (bool): Solve LP-relaxation if False; otherwise, solve IP.
    '''
    ITEMS = items.keys()
    BINS = range(sum(items.values())) # one item per bin
    v = volume
    count = items
    
    # define model
    m = gp.Model("pack_bins")
    
    # decision variables
    
    x = {} # the number of items of size i that are put in bin j
    for i in ITEMS:
        for j in BINS:
            if integer:
                x[i,j] = m.addVar(vtype=GRB.INTEGER, lb=0, ub=GRB.INFINITY, name=('(i%s, b%s)' % (i,j)))
            else:
                x[i,j] = m.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY, name=('(i%s, b%s)' % (i,j)))
                
    y = {} # 1 if bin i is used; 0 otherwise
    for i in BINS:
        if integer:
            y[i] = m.addVar(vtype=GRB.BINARY, name=('%s' % (i)))
        else:
            y[i] = m.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name=('%s' % (i)))
            
     # objective function
    m.setObjective(sum(y[i] for i in BINS), GRB.MINIMIZE)
    
    # constraints
    
    # subject to: bin volume
    for j in BINS:        
        m.addConstr(sum(x[i,j]*i for i in ITEMS) <= v, 'volume_%d' % (j))
        
    # subject to: all items packed
    for i in ITEMS:
        m.addConstr(sum(x[i,j] for j in BINS) == count[i], 'packed_%d' % (i))
        
    # subject to: bin used or not
    for j in BINS:
        M = v
        m.addConstr(sum(x[i,j]*i for i in ITEMS) <= M*y[j], 'used_%d' % (j))
    
    m.optimize()

In [ ]:
#PerfectBinPacking(mean=35, std=5, n=50, v=150)
with open('data-bin_packing/items_168s.pickle', 'rb') as handle:
    items = pickle.load(handle)

In [ ]:
PackBins(items, volume=150)

In [ ]:
PackBins(items, volume=150, integer=True)